## Part 1

In [1]:
TEST_INFILE = "inputs/day_21_test_1.txt"
INFILE = "inputs/day_21_1.txt"

#with open(TEST_INFILE) as infile:
with open(INFILE) as infile:
    lines = infile.read().splitlines()

grid = [list(line) for line in lines]

In [2]:
from enum import Enum

class Point:
    def __init__(self, row, col):
        self.row = row
        self.col = col

    def manhattan_distance(self, other):
        return abs(self.row - other.row) + abs(self.col - other.col)

    def __repr__(self):
        return f"({self.row}, {self.col})"

    def __add__(self, other):
        return Point(self.row + other.row, self.col + other.col)
    
    def __sub__(self, other):
        return Point(self.row - other.row, self.col - other.col)
    
    def __mul__(self, number):
        return Point(self.row * number, self.col * number)

    def __radd__(self, other):
        return self + other
    
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    def __hash__(self):
        return hash((self.row, self.col))
    

class Direction(Enum):
    UP = Point(-1, 0)
    DOWN = Point(1, 0)
    LEFT = Point(0, -1)
    RIGHT = Point(0, 1)


assert Point(0, 0) == Point(0, 0)
assert Point(1, 0) + Point(1, 10) == Point(2, 10)
p = Point(1, 0)
p += Point(1, 10)
assert p == Point(2, 10)
assert Point(1, 2) * 10 == Point(10, 20)
assert Point(20, 10) - Point(5, 5) == Point(15, 5)

In [3]:
START = [Point(row_num, col_num) if symbol=="S" else None for row_num, row in enumerate(grid) for col_num, symbol in enumerate(row)]
START = list(filter(lambda x: x is not None, START))[0]
START

(65, 65)

In [4]:
def get_neighbors(pos, grid=grid):
    neighbors = []
    for direction in Direction:
        neighbor = pos + direction.value
        if neighbor.row < 0 or neighbor.row >= len(grid):
            continue
        if neighbor.col < 0 or neighbor.col >= len(grid[0]):
            continue
        if grid[neighbor.row][neighbor.col] != '#':
            neighbors.append(neighbor)
    return neighbors

In [5]:
def print_grid(occupied: set, grid=grid):
    for row_num, row in enumerate(grid):
        for col_num, symbol in enumerate(row):
            if Point(row_num, col_num) in occupied:
                print("O", end="")
            else:
                print(symbol, end="")
        print("")

In [6]:
def take_step(frontier, grid=grid):
    new_frontier = set()
    for pos in frontier:
        neighbors = get_neighbors(pos, grid)
        for neighbor in neighbors:
            new_frontier.add(neighbor)
    return new_frontier

In [7]:
frontier = set([START])
for _ in range(64):
    frontier = take_step(frontier)
print_grid(frontier)

...................................................................................................................................
...................#.....#.....#.#.............#.#......#........O..............#......#.......#......#...#...#.##..........##.....
......#......#.....#....#........#...............#..............O.O.........#................#...........#...#......#.....#.##.#...
..#.........#......#........##..##.#............#.#.#.#........O.O.O......##....#......#.#............#...........#................
.#...............#....###.#....#...........#........##........O.O.O.O...............#...............##.............#....#..........
............#.............##..#.#.....#..........#...#.......O.O.O.O.O....###...#.......#...........#.......##.#.........#.........
.................#..........#..#..#.....##..#........#......O.O.#.O.O.O......#...##.#..............#.....#.........#....#..#.......
...#.......#..........#..#......###............#...#.#.....O.O.O.O.O.O.O....

In [8]:
len(frontier)

3764

## Part 2

In [9]:
# https://github.com/villuna/aoc23/wiki/A-Geometric-solution-to-advent-of-code-2023,-day-21

In [10]:
# horizontal alley from the start
all([c in ["S", "."] for c in grid[START.row]])

True

In [11]:
# vertical alley from the start
all([grid[row][START.col] in ["S", "."] for row in range(len(grid))])

True

In [12]:
N_ROWS, N_COLS = len(grid), len(grid[0])

In [13]:
START, N_ROWS, N_COLS

((65, 65), 131, 131)

In [ ]:
def get_neighbors_infinite(pos, grid=grid):
    neighbors = []
    for direction in Direction:
        neighbor = pos + direction.value
        if grid[neighbor.row % len(grid)][neighbor.col % len(grid[0])] != '#':
            neighbors.append(neighbor)
    return neighbors


def take_step_infinite(frontier, grid=grid):
    new_frontier = set()
    for pos in frontier:
        neighbors = get_neighbors_infinite(pos, grid)
        for neighbor in neighbors:
            new_frontier.add(neighbor)
    return new_frontier


def print_grid_infinite(occupied: set, grid=grid):
    min_row = min(p.row for p in occupied)
    max_row = max(p.row for p in occupied)
    min_col = min(p.col for p in occupied)
    max_col = max(p.row for p in occupied)

    for row_num in range(min_row, max_row + 1):
        for col_num in range(min_col, max_col + 1):
            if Point(row_num, col_num) in occupied:
                print("O", end="")
            else:
                symbol = grid[row_num % len(grid)][col_num % len(grid[0])]
                print(symbol, end="")
        print("")

In [27]:
#STEPS = 26_501_365
# 1 tile exactly
STEPS = 65 + 131
# 2 tiles exactly
# STEPS = 65 + 2*131
# 3 tile exactly
#STEPS = 65 + 3*131

STEPS_TO_EDGE = (N_COLS - 1 - START.col)
# diamond where the edges are 202300 full tiles away
N_TILES = (STEPS - STEPS_TO_EDGE) / N_COLS
N_TILES

1.0

In [28]:
brute_force_frontier = set([START])
for _ in range(STEPS):
    brute_force_frontier = take_step_infinite(brute_force_frontier)
print_grid_infinite(brute_force_frontier)

....................................................................................................................................................................................................O....................................................................................................................................................................................................
...................#.....#.....#.#.............#.#......#.......................#......#.......#......#...#...#.##..........##........................#.....#.....#.#.............#.#......#.......O.O.............#......#.......#......#...#...#.##..........##........................#.....#.....#.#.............#.#......#.......................#......#.......#......#...#...#.##..........##.....
......#......#.....#....#........#...............#..........................#................#...........#...#......#.....#.##.#.........#......#.....#....#........#...............#.............O.O.O........#....

In [29]:
len(brute_force_frontier)

34441

In [30]:
# for a given point, find the shortest path from the start to the point on the grid
def shortest_paths(grid=grid):
    dists = {}
    dists[START] = 0
    queue = [(0, START)]
    while len(queue) > 0:
        steps, current = queue.pop(0)
        for neighbor in get_neighbors(current, grid):
            if neighbor not in dists or (steps + 1) < dists[neighbor]:
                dists[neighbor] = steps + 1
                queue.append((steps + 1, neighbor))
    return dists

In [31]:
dists = shortest_paths(grid)

In [32]:
rocks = [grid[r][c] for r in range(len(grid)) for c in range(len(grid[0])) if grid[r][c] == "#"]
# we end up with 5 unreachable points that are truly unreachable
len(dists), N_ROWS * N_COLS - len(rocks)

(15221, 15226)

In [34]:
#count

In [36]:
even_corners = len([pos for pos, steps in dists.items() if steps > STEPS_TO_EDGE and steps % 2 == 0])
odd_corners  = len([pos for pos, steps in dists.items() if steps > STEPS_TO_EDGE and steps % 2 == 1])
even_full    = len([pos for pos, steps in dists.items() if steps % 2 == 0])
odd_full     = len([pos for pos, steps in dists.items() if steps % 2 == 1])

In [37]:
total = (N_TILES+1)**2 * odd_full + N_TILES**2 * even_full - (N_TILES + 1) * odd_corners + N_TILES * even_corners

In [39]:
# we end up somehow with one too many points for n=1
int(total), len(brute_force_frontier)

(34442, 34441)

In [86]:
bf_ur = set([Point(pos.row + len(grid) , pos.col - len(grid[0])) for pos in brute_force_frontier if pos.row < 0 and pos.col >= len(grid[0])])
bf_lr = set([Point(pos.row - len(grid), pos.col - len(grid[0])) for pos in brute_force_frontier if pos.row >= len(grid) and pos.col >= len(grid[0])])
bf_ul = set([Point(pos.row + len(grid), pos.col + len(grid[0])) for pos in brute_force_frontier if pos.row < 0 and pos.col < 0])
bf_ll = set([Point(pos.row - len(grid), pos.col + len(grid[0])) for pos in brute_force_frontier if pos.row >= len(grid) and pos.col < 0])
calc_even_corners = set([pos for pos, steps in dists.items() if steps > STEPS_TO_EDGE and steps % 2 == 0])
calc_odd_corners = set([pos for pos, steps in dists.items() if steps > STEPS_TO_EDGE and steps % 2 == 1])

In [96]:
missing = calc_even_corners.difference(bf_ur.union(bf_lr).union(bf_ul).union(bf_ll))

In [106]:
# would be in top right
missing = list(missing)[0]

In [112]:
missing

(6, 76)

In [113]:
dists[Point(missing.row, missing.col)]

70

In [111]:
Point(missing.row + 2*len(grid), len(grid[0]) - (missing.col - len(grid[0])))

(268, 186)

In [115]:
min_row = min(p.row for p in bf_ll)
max_row = max(p.row for p in bf_ll)
min_col = min(p.col for p in bf_ll)
max_col = max(p.col for p in bf_ll)
for row_num in range(min_row, max_row + 1):
    for col_num in range(min_col, max_col + 1):
        if Point(row_num, col_num) in bf_ll:
            print("O", end="")
        else:
            symbol = grid[row_num % len(grid)][col_num % len(grid[0])]
            print(symbol, end="")
    print("")

O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O
.O.O.O.O.O.O.O#O.O.O.#.O.O.O.#.O.O.O#O.O#O.O#O##.O.O.O.O.O##.O.O.
..O.O.O.O.#.O.O.O.O.O.O.O.O#O.O.O.O.O.O#O.O#O.O.O.#.O.O.#.##O#O.O
...O.O.O##.O.O#O.O.O.#.#.O.O.O.O.O.O#O.O.O.O.O.O#O.O.O.O.O.O.O.O.
....O.O.O.O.O.O.O.#.O.O.O.O.O.O.O.##O.O.O.O.O.O.O#O.O.#.O.O.O.O.O
.....O.O###O.O#O.O.O.O#O.O.O.O.O.O#O.O.O.O##.#.O.O.O.O.#.O.O.O.O.
......O.O..#O.O##.#.O.O.O.O.O.O.O#O.O.O#O.O.O.O.O#O.O.#.O#O.O.O.O
.......O.O#O.O.O#O.O.O.O.###.#.O.O.O.O.O.O.#.O#O.O.#.O.O.O.O.O#O.
........O.O.#.#.O.O.O.O.O.O.O.O.O##.O.#.#.#.O.O#O.O##.O.O.O.O.O#O
.........O.O.O.#.O#O.O.O.O.O.O.O#O#O.O.O.O.O.O#O.O.#.O.O.O.O#O.O.
#.........O.O.O#O.O.O.O#O.O.O.O.O.O.O.O.O.O#O.O.O.O#O.O##.O#O.O#O
..#........O.O.O.O.#.O.###.#.O.#.O.O.O.O.O#O##.O.#.O.O.O.O.O.O.O.
............O.O.O.O.O.O.#.#.O.O.O.O.O.O.O.O.O.O.O.#.O.O.O.O.O.O#O
.............O.O.O.#.O.O.O.O.O.O.O.O.O.O.O.O#O.#.O#O.O.O.O.O.O#O.
....#..#......O.O.#.O.O#O#O.O.O.O#O.O#O#O.O#O.O.O.O#O#O.O.O.O.#.O
.#.......#

In [119]:
min_row = min(p.row for p in calc_even_corners)
max_row = max(p.row for p in calc_even_corners)
min_col = min(p.col for p in calc_even_corners)
max_col = max(p.col for p in calc_even_corners)
for row_num in range(min_row, max_row + 1):
    #if row_num < 65:
    #    continue
    for col_num in range(min_col, max_col + 1):
        if col_num <= 65:
            continue
        if Point(row_num, col_num) in calc_even_corners:
            print("O", end="")
        else:
            symbol = grid[row_num % len(grid)][col_num % len(grid[0])]
            print(symbol, end="")
    print("")

O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O
.O.O.O.O.O.O.O#O.O.O.#.O.O.O.#.O.O.O#O.O#O.O#O##.O.O.O.O.O##.O.O.
..O.O.O.O.#.O.O.O.O.O.O.O.O#O.O.O.O.O.O#O.O#O.O.O.#.O.O.#.##O#O.O
...O.O.O##.O.O#O.O.O.#.#.O.O.O.O.O.O#O.O.O.O.O.O#O.O.O.O.O.O.O.O.
....O.O.O.O.O.O.O.#.O.O.O.O.O.O.O.##O.O.O.O.O.O.O#O.O.#.O.O.O.O.O
.....O.O###O.O#O.O.O.O#O.O.O.O.O.O#O.O.O.O##.#.O.O.O.O.#.O.O.O.O.
......O.O.O#O.O##.#.O.O.O.O.O.O.O#O.O.O#O.O.O.O.O#O.O.#.O#O.O.O.O
.......O.O#O.O.O#O.O.O.O.###.#.O.O.O.O.O.O.#.O#O.O.#.O.O.O.O.O#O.
........O.O.#.#.O.O.O.O.O.O.O.O.O##.O.#.#.#.O.O#O.O##.O.O.O.O.O#O
.........O.O.O.#.O#O.O.O.O.O.O.O#O#O.O.O.O.O.O#O.O.#.O.O.O.O#O.O.
#.........O.O.O#O.O.O.O#O.O.O.O.O.O.O.O.O.O#O.O.O.O#O.O##.O#O.O#O
..#........O.O.O.O.#.O.###.#.O.#.O.O.O.O.O#O##.O.#.O.O.O.O.O.O.O.
............O.O.O.O.O.O.#.#.O.O.O.O.O.O.O.O.O.O.O.#.O.O.O.O.O.O#O
.............O.O.O.#.O.O.O.O.O.O.O.O.O.O.O.O#O.#.O#O.O.O.O.O.O#O.
....#..#......O.O.#.O.O#O#O.O.O.O#O.O#O#O.O#O.O.O.O#O#O.O.O.O.#.O
.#.......#